In [9]:
import numpy as np
import pandas as pd
import ast

file_path = "/kaggle/input/binancedata2/TRADES_CopyTr_90D_ROI.csv"  # Replace with your file path
df = pd.read_csv(file_path)

In [10]:
def safe_eval(row):
    try:
        return ast.literal_eval(row)
    except (ValueError, SyntaxError):
        return None

df["Trade_History"] = df["Trade_History"].apply(safe_eval)

In [11]:
# Explode and normalize Trade_History
df_exploded = df.explode("Trade_History")
trade_data = pd.json_normalize(df_exploded["Trade_History"])

In [12]:
# Combine with original dataset columns
df_combined = pd.concat([df_exploded.drop(columns=["Trade_History"]).reset_index(drop=True), trade_data.reset_index(drop=True)], axis=1)
df_combined

,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,3925368433214965504,1.718900e+12,SOLUSDT,BUY,132.537000,-0.994027,USDT,1988.055000,USDT,0.0,USDT,SOL,15.0,LONG,True
1,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.121820,-0.279796,USDT,1398.980880,USDT,0.0,USDT,DOGE,11484.0,LONG,False
2,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.121820,-0.039494,USDT,197.470220,USDT,0.0,USDT,DOGE,1621.0,LONG,False
3,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.121820,-0.008284,USDT,16.567520,USDT,0.0,USDT,DOGE,136.0,LONG,True
4,3925368433214965504,1.718900e+12,DOGEUSDT,BUY,0.121820,-0.046109,USDT,92.217740,USDT,0.0,USDT,DOGE,757.0,LONG,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211273,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035968,-0.025070,USDT,50.139392,USDT,0.0,USDT,JASMY,1394.0,BOTH,False
211274,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035968,-0.008255,USDT,16.509312,USDT,0.0,USDT,JASMY,459.0,BOTH,False
211275,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035969,-0.031545,USDT,63.089626,USDT,0.0,USDT,JASMY,1754.0,BOTH,False
211276,3768170840939476993,1.718550e+12,JASMYUSDT,SELL,0.035969,-0.031905,USDT,63.809006,USDT,0.0,USDT,JASMY,1774.0,BOTH,False


In [13]:
# Financial Metrics Calculations
# ROI
df_combined["ROI"] = (df_combined["realizedProfit"] / df_combined["quantity"]) * 100

# PnL (Profit and Loss)
pnl = df_combined.groupby("Port_IDs")["realizedProfit"].sum()

# Sharpe Ratio
df_combined["returns"] = df_combined["realizedProfit"] / df_combined["quantity"]
sharpe_ratio = df_combined.groupby("Port_IDs")["returns"].mean() / df_combined.groupby("Port_IDs")["returns"].std()

# Maximum Drawdown (MDD)
df_combined["cumulative_returns"] = df_combined.groupby("Port_IDs")["realizedProfit"].cumsum()
drawdown = df_combined.groupby("Port_IDs")["cumulative_returns"].apply(
    lambda x: (x / x.cummax() - 1).min()
)

# Win Rate
df_combined["win"] = df_combined["realizedProfit"] > 0
win_rate = df_combined.groupby("Port_IDs")["win"].mean() * 100

# Total Positions
total_positions = df_combined.groupby("Port_IDs").size()

# Win Positions
win_positions = df_combined[df_combined["win"]].groupby("Port_IDs").size()


In [14]:
# Combine Metrics into a DataFrame
metrics = pd.DataFrame({
    "PnL": pnl,
    "Sharpe Ratio": sharpe_ratio,
    "Max Drawdown (MDD)": drawdown,
    "Win Rate (%)": win_rate,
    "Total Positions": total_positions,
    "Win Positions": win_positions,
})
# Replace NaN with 0 for metrics where applicable
metrics = metrics.fillna(0)

In [15]:
# Ranking Algorithm
weights = {"PnL": 0.4, "Sharpe Ratio": 0.3, "Win Rate (%)": 0.2, "Win Positions": 0.1}
metrics["Score"] = sum(metrics[col] * weight for col, weight in weights.items())
metrics["Rank"] = metrics["Score"].rank(ascending=False)

In [16]:
# Sort by Rank
metrics = metrics.sort_values("Rank", ascending=True)

In [17]:
# Save Deliverables
metrics.to_csv("calculated_metrics.csv", index=True)
top_20 = metrics.head(20)
top_20.to_csv("top_20_accounts.csv", index=True)

In [18]:
# Output
print("Analysis Complete!")
print("Top 20 Accounts:")
print(top_20)

Analysis Complete!
Top 20 Accounts:
                              PnL  Sharpe Ratio  Max Drawdown (MDD)  \
Port_IDs                                                              
4020204877254599680  71998.855953     -0.088411           -8.028348   
3999240873283311617  42574.473679      0.591529           -0.052816   
4021669203289716224  26427.331592      0.040739           -0.161568   
3960874214179953664  19567.471286      0.116708          -34.536210   
3907081197088384000  18015.997370      0.784527           -0.242908   
4028701921959171840  17601.401398      0.674660           -0.045177   
3956076827719377409  16790.012238      0.680371           -0.246143   
3986814617275053313  16337.461881      1.288552           -0.000149   
3788465932399412480  13960.966457      0.217427           -0.224263   
4022565861939831809  14197.577076      0.597009           -0.108960   
3987739404272887297  12464.606315      0.151775           -0.310502   
4008711265867865600  11298.706390      0.